In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
df= pd.read_csv('./filetorch.csv',index_col=[0])

In [3]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,6,189615,155782,2016-04-21 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [4]:
del df['date']

In [5]:
df.head()

,id,player_fifa_api_id,player_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1,2,218353,505942,67.0,71.0,right,medium,medium,49.0,44.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,62.0,66.0,right,medium,medium,49.0,44.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,61.0,65.0,right,medium,medium,48.0,43.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,61.0,65.0,right,medium,medium,48.0,43.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,6,189615,155782,74.0,76.0,left,high,medium,80.0,53.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [6]:
del df['player_fifa_api_id']
del df['id']
del df['player_api_id']

In [7]:
df.isnull().any()

overall_rating         False
potential              False
preferred_foot         False
attacking_work_rate    False
defensive_work_rate    False
crossing               False
finishing              False
heading_accuracy       False
short_passing          False
volleys                False
dribbling              False
curve                  False
free_kick_accuracy     False
long_passing           False
ball_control           False
acceleration           False
sprint_speed           False
agility                False
reactions              False
balance                False
shot_power             False
jumping                False
stamina                False
strength               False
long_shots             False
aggression             False
interceptions          False
positioning            False
vision                 False
penalties              False
marking                False
standing_tackle        False
sliding_tackle         False
gk_diving              False
gk_handling   

In [8]:
df2=df[0:500]

In [9]:
len(df2)

499

In [10]:
df2.head()

,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,74.0,76.0,left,high,medium,80.0,53.0,58.0,71.0,40.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [11]:
# Get list of categorical variables
s = (df2.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['preferred_foot', 'attacking_work_rate', 'defensive_work_rate']


In [12]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
df3 = pd.DataFrame(OH_encoder.fit_transform(df2[object_cols]))
df3.index = df2.index
# Remove categorical columns (will replace with one-hot encoding)
num_X_train = df2.drop(object_cols, axis=1)
# Add one-hot encoded columns to numerical features
df3 = pd.concat([num_X_train, df3], axis=1)

In [13]:
df3.head(5)

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,gk_kicking,gk_positioning,gk_reflexes,0,1,2,3,4,5,6
1,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,10.0,8.0,8.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,62.0,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,...,10.0,8.0,8.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,...,9.0,7.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,...,9.0,7.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
5,74.0,76.0,80.0,53.0,58.0,71.0,40.0,73.0,70.0,69.0,...,9.0,9.0,12.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [14]:
df3.T

,1,2,3,4,5,6,7,8,9,10,...,519,520,521,522,523,524,525,526,527,528
overall_rating,67.0,62.0,61.0,61.0,74.0,74.0,73.0,73.0,73.0,73.0,...,67.0,67.0,68.0,66.0,53.0,53.0,64.0,64.0,63.0,63.0
potential,71.0,66.0,65.0,65.0,76.0,76.0,75.0,75.0,75.0,77.0,...,76.0,82.0,82.0,82.0,74.0,74.0,70.0,70.0,69.0,66.0
crossing,49.0,49.0,48.0,48.0,80.0,80.0,79.0,79.0,79.0,79.0,...,61.0,61.0,61.0,55.0,41.0,41.0,35.0,35.0,34.0,34.0
finishing,44.0,44.0,43.0,43.0,53.0,53.0,52.0,51.0,51.0,51.0,...,49.0,49.0,49.0,50.0,50.0,50.0,30.0,30.0,29.0,29.0
heading_accuracy,71.0,71.0,70.0,70.0,58.0,58.0,57.0,57.0,57.0,57.0,...,66.0,66.0,66.0,66.0,48.0,48.0,56.0,56.0,55.0,55.0
short_passing,61.0,61.0,60.0,60.0,71.0,71.0,70.0,70.0,70.0,70.0,...,74.0,74.0,74.0,73.0,62.0,62.0,57.0,57.0,56.0,56.0
volleys,44.0,44.0,43.0,43.0,40.0,32.0,29.0,29.0,29.0,29.0,...,62.0,62.0,62.0,62.0,62.0,62.0,32.0,32.0,31.0,31.0
dribbling,51.0,51.0,50.0,50.0,73.0,73.0,71.0,71.0,71.0,71.0,...,70.0,70.0,70.0,69.0,54.0,54.0,36.0,36.0,35.0,35.0
curve,45.0,45.0,44.0,44.0,70.0,70.0,68.0,68.0,68.0,68.0,...,65.0,65.0,65.0,65.0,65.0,65.0,27.0,27.0,26.0,26.0
free_kick_accuracy,39.0,39.0,38.0,38.0,69.0,69.0,69.0,69.0,69.0,69.0,...,43.0,43.0,43.0,43.0,43.0,43.0,30.0,30.0,29.0,29.0


In [15]:
import torch
import matplotlib.pyplot as plt
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [16]:
columns = df3.columns

In [17]:
scaler = MinMaxScaler()
df3 = pd.DataFrame(scaler.fit_transform(df3), columns = columns)
df3.head()

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,gk_kicking,gk_positioning,gk_reflexes,0,1,2,3,4,5,6
0,0.540541,0.441176,0.528571,0.453125,0.833333,0.612903,0.447761,0.455696,0.476923,0.333333,...,0.067568,0.044444,0.042553,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.405405,0.294118,0.528571,0.453125,0.833333,0.612903,0.447761,0.455696,0.476923,0.333333,...,0.067568,0.044444,0.042553,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.378378,0.264706,0.514286,0.437500,0.818182,0.596774,0.432836,0.443038,0.461538,0.317460,...,0.054054,0.022222,0.021277,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,0.378378,0.264706,0.514286,0.437500,0.818182,0.596774,0.432836,0.443038,0.461538,0.317460,...,0.054054,0.022222,0.021277,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.729730,0.588235,0.971429,0.593750,0.636364,0.774194,0.388060,0.734177,0.861538,0.809524,...,0.054054,0.066667,0.127660,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [18]:
y=df3['overall_rating']

In [19]:
del df3['overall_rating']

In [20]:

X=df3

In [21]:
X.head()

,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,...,gk_kicking,gk_positioning,gk_reflexes,0,1,2,3,4,5,6
0,0.441176,0.528571,0.453125,0.833333,0.612903,0.447761,0.455696,0.476923,0.333333,0.707692,...,0.067568,0.044444,0.042553,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.294118,0.528571,0.453125,0.833333,0.612903,0.447761,0.455696,0.476923,0.333333,0.707692,...,0.067568,0.044444,0.042553,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.264706,0.514286,0.437500,0.818182,0.596774,0.432836,0.443038,0.461538,0.317460,0.692308,...,0.054054,0.022222,0.021277,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,0.264706,0.514286,0.437500,0.818182,0.596774,0.432836,0.443038,0.461538,0.317460,0.692308,...,0.054054,0.022222,0.021277,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.588235,0.971429,0.593750,0.636364,0.774194,0.388060,0.734177,0.861538,0.809524,0.769231,...,0.054054,0.066667,0.127660,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [22]:
X=X.values
y=y.values

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=324)

In [24]:
#Define training hyperprameters.
batch_size = 50
num_epochs = 200
learning_rate = 0.01
size_hidden= 100

#Calculate some other hyperparameters based on data.  
batch_no = len(X_train) // batch_size  #batches
cols=X_train.shape[1] #Number of columns in input matrix
n_output=1


In [25]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, size_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(cols, size_hidden)   # hidden layer
        self.predict = torch.nn.Linear(size_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x
net = Net(cols, size_hidden, n_output)

In [26]:
#Adam is a specific flavor of gradient decent which is typically better
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
criterion = torch.nn.MSELoss(size_average=False)  # this is for regression mean squared loss

C:\Users\Odeyemi Tolutope\Anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [28]:
from sklearn.utils import shuffle
from torch.autograd import Variable
running_loss = 0.0
for epoch in range(num_epochs):
    #Shuffle just mixes up the dataset between epocs
    X_train, y_train = shuffle(X_train, y_train)
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(X_train[start:end]))
        labels = Variable(torch.FloatTensor(y_train[start:end]))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print("outputs",outputs)
        #print("outputs",outputs,outputs.shape,"labels",labels, labels.shape)
        loss = criterion(outputs, torch.unsqueeze(labels,dim=1))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('Epoch {}'.format(epoch+1), "loss: ",running_loss)
    running_loss = 0.0



Epoch 1 loss:  22.841266751289368
Epoch 2 loss:  7.32373309135437
Epoch 3 loss:  5.868932127952576
Epoch 4 loss:  4.53240430355072
Epoch 5 loss:  4.091571152210236
Epoch 6 loss:  3.6777701675891876
Epoch 7 loss:  3.4236938655376434
Epoch 8 loss:  3.018176853656769
Epoch 9 loss:  2.73531574010849
Epoch 10 loss:  2.319806456565857
Epoch 11 loss:  1.904480442404747
Epoch 12 loss:  1.9158119410276413
Epoch 13 loss:  1.6753379255533218
Epoch 14 loss:  1.496922105550766
Epoch 15 loss:  1.4093086570501328
Epoch 16 loss:  1.2569571733474731
Epoch 17 loss:  1.1391529738903046
Epoch 18 loss:  1.0520550236105919
Epoch 19 loss:  0.9909539967775345
Epoch 20 loss:  1.0525892823934555
Epoch 21 loss:  0.9729135185480118
Epoch 22 loss:  1.111464075744152
Epoch 23 loss:  1.0712106749415398
Epoch 24 loss:  1.215196892619133
Epoch 25 loss:  0.9446417987346649
Epoch 26 loss:  0.7487944588065147
Epoch 27 loss:  0.7485508099198341
Epoch 28 loss:  0.7459779977798462
Epoch 29 loss:  0.6819637417793274
Epoch 30

In [30]:
from sklearn.metrics import r2_score

X = Variable(torch.FloatTensor(X_train)) 
result = net(X)
pred=result.data[:,0].numpy()
print(len(pred),len(y_train))
r2_score(pred,y_train)

334 334


0.9702516193741156

In [31]:
import pandas as pd
from sklearn.metrics import r2_score
#This is a little bit tricky to get the resulting prediction.  
def calculate_r2(x,y=[]):
    """
    This function will return the r2 if passed x and y or return predictions if just passed x. 
    """
    # Evaluate the model with the test set. 
    X = Variable(torch.FloatTensor(x))  
    result = net(X) #This outputs the value for regression
    result=result.data[:,0].numpy()
  
    if len(y) != 0:
        r2=r2_score(result, y)
        print("R-Squared", r2)
        #print('Accuracy {:.2f}'.format(num_right / len(y)), "for a total of ", len(y), "records")
        return pd.DataFrame(data= {'actual': y, 'predicted': result})
    else:
        print("returning predictions")
        return result


In [32]:
result1=calculate_r2(X_train,y_train)
result2=calculate_r2(X_test,y_test)

R-Squared 0.9702516193741156
R-Squared 0.9668357160293894
